In [1]:
import networkx as nx
import wikipediaapi
#import graphviz

In [2]:
user_agent = 'wikipedia-graph (danielwarkus@hotmail.com)'

In [3]:
wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language='de')

In [4]:
python_page = wiki.page('Python (Programmiersprache)')

In [10]:
section = python_page.section_by_title("Ausnahmebehandlung")

In [13]:
print(type(section.text))

<class 'str'>


In [9]:
excluded_namespaces = ['Benutzer:', 'Wikipedia:', 'Hilfe:', 'Portal:', 
                          'Kategorie:', 'MediaWiki:', 'Vorlage:', 'Spezial:', 'Datei:', 'Modul:', 'Wikipedia Diskussion:', 'Kategorie Diskussion:', 'Hilfe Diskussion:', 'Portal Diskussion:', 'Benutzer Diskussion:', 'MediaWiki Diskussion:', 'Vorlage Diskussion:', 'Modul Diskussion:', 'Datei Diskussion:', 'Spezial:', 'Diskussion:']

In [10]:
def get_backlinks(page):
    backlinks = []
    for link in page.backlinks.values():
        # Prüfe ob der Link-Titel mit einem der ausgeschlossenen Namespaces beginnt
        if not any(link.title.startswith(ns) for ns in excluded_namespaces):
            backlinks.append(link.title)
            
    return backlinks

In [11]:
def get_links(page):
    links = []
    for link in page.links.values():
        if not any(link.title.startswith(ns) for ns in excluded_namespaces):
            links.append(link.title)
    return links

In [12]:
def get_page(link):
    return wiki.page(link)

In [13]:
def make_graph(page, depth):
    graph = nx.DiGraph()
    links = get_links(page)
    for link in links:
        graph.add_edge(page.title, link)
        if depth > 0:
            next_page = get_page(link)
            graph = nx.compose(graph, make_graph(next_page, depth-1))
    return graph

Optimieren! Liste von Knoten -> Subgraph

In [14]:
def make_graph_optimized(page):
    graph = nx.DiGraph()
    page_links = get_links(page)
    
    for link in page_links:
        graph.add_edge(page.title, link)
        
        new_page = get_page(link)
        new_page_links = get_links(new_page)
        for link in new_page_links:
            graph.add_edge(new_page.title, link)
    return graph

In [15]:
def make_backgraph(page):
    backgraph = nx.DiGraph()
    backgraph.add_node(page.title)
    backlinks = get_backlinks(page)
    for link in backlinks:
        backgraph.add_edge(page.title, link)
        new_page = get_page(link)
        new_page_links = get_links(new_page)
       
        for link in new_page_links:
            backgraph.add_edge(new_page.title, link)
  
    return backgraph

# Erstelle den Ursprungsgraphen
### Hier wird ein Graph erstellt, der alle Links (und deren Links) von der Python-Seite enthält.
### Die Artikelnamen sind dabei die Knoten und die Verlinkungen die Kanten. 

In [16]:
#graph = make_graph(python_page, 1)

In [17]:
graph = make_graph_optimized(python_page)

In [18]:
backlinks = get_backlinks(python_page)

In [19]:
print(backlinks)

['Ada (Programmiersprache)', 'C (Programmiersprache)', 'C++', 'Garbage Collection', 'INTERCAL', 'JavaScript', 'Julianisches Datum', 'Kommandozeileninterpreter', 'Lisp', 'Linux', 'LaTeX', 'Monty Python', 'Message-Digest Algorithm 5', 'Makroökonomie', 'Apache OpenOffice', 'Zeittafel der Programmiersprachen', 'Java (Programmiersprache)', 'Perl (Programmiersprache)', 'Quelltext', 'Regulärer Ausdruck', 'Rekursion', 'Scalable Vector Graphics', 'Sprache', 'Smalltalk (Programmiersprache)', 'Skriptsprache', 'Tcl', 'World Wide Web', 'Extensible Markup Language', 'XSL Transformation', 'Zwischenablage', 'Zeichenkette', 'At-Zeichen', 'Python', 'Tk (Toolkit)', 'Gnome', 'CAD', 'Dijkstra-Algorithmus', 'Potenz (Mathematik)', 'Freie Software', 'Interpreter', 'Präsentationsprogramm', 'Vollständiger Graph', 'Isomorphie von Graphen', 'Qt (Bibliothek)', 'KDevelop', 'GTK (Programmbibliothek)', 'Make', 'Stapelspeicher', 'Webdesign', 'Website', 'Samba (Software)', 'Microsoft Foundation Classes', 'Softwareentwi

In [20]:
print(len(python_page.links), len(backlinks), (len(backlinks)/len(python_page.links)))

212 999 4.712264150943396


In [21]:
backgraph = make_backgraph(python_page)

In [22]:
complete_graph = nx.compose(graph, backgraph)
# ANMERKUNG: ZUM TEST WURDE COMPLETE_GRAPH VERWENDET;; SONST NUR GRAPH

In [23]:
# from graph, export edges as a tuple to a csv file
edges = graph.edges
with open('edges_original.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

# Versuchserklärung
#### Ziel ist es, Artikelvorschläge zu generieren, die thematisch zu Python passen. Dabei werde ich 4 verschiedene Ansätze testen (dabei wird immer der höchste Grad der Knoten betrachtet):
#### 1. Extrahiere die Knoten mit dem höchsten Grad (ungefiltert)
#### 2. Betrachte nur die direkten Nachfolger von Python
#### 3. Betrachte nur die Knoten, die zu Python verlinkt sind
#### 4. Betrachte nur die direkten Nachfolger von Python, die auch eine Verbindung zurück zu Python haben (beide Bedingungen)

#### Die Tests 2-4 wurde ich sowohl am Hauptgraphen, als auch an einem Subgraphen durchführen, um zu schauen ob dies ein Unterschied macht.

# Versuch 1: Extrahiere die Knoten mit dem höchsten Grad
#### Im ersten Versuch werden die Knoten mit dem höchsten Grad extrahiert. Man kann erkennen, dass die Artikelvorschläge zu oberflächlich sind. 

In [24]:
# print the top 3 nodes with the highest degree
top_5_nodes = sorted(complete_graph.degree, key=lambda x: x[1], reverse=True)[:5]
print(top_5_nodes)

[('Python (Programmiersprache)', 2096), ('Betriebssystem', 825), ('Unicode', 728), ('Amsterdam', 671), ('Linux', 657)]


# Versuch 2: Extrahiere die direkten Nachfolger von Python
#### Um die Auswahl passender Artikel einzuschränken, werden nur die direkten Nachfolger von Python betrachtet.
#### Diese neue Bedingung soll dazu führen, nur Artikel in Betracht zu ziehen, die auch von Python refenziert werden.

#### Getestet habe ich diese Bedinung an zwei Graphen:
#### 1. Der ursprüngliche Graph
#### 2. Ein Subgraph, der nur die Knoten enthält, die von Python referenziert werden.
 

In [25]:
links_python = get_links(python_page)

In [26]:
reduced_node_list = [node for node in complete_graph.nodes() if node in links_python]

## Erstelle Subgraphen und extrahiere die Knoten mit dem höchsten Grad
#### Hier wird ein Subgraph erstellt, der nur die Knoten enthält, die direkte Nachfolger von Python sind.
#### Die Ergebnisse sind besser, als die des ursprünglichen Graphen.

In [27]:
# create a subgraph
subgraph = complete_graph.subgraph(reduced_node_list)

In [28]:
top_5_reduced_nodes = sorted(subgraph.degree(), key=lambda x: x[1], reverse=True)[:5]
print(top_5_reduced_nodes)

[('Java (Programmiersprache)', 112), ('C++', 110), ('Betriebssystem', 105), ('C (Programmiersprache)', 102), ('Perl (Programmiersprache)', 76)]


## Extrahiere Knoten mit dem höchsten Grad aus Hauptgraphen
#### Hier zeigt die Zusatzbedingung keine Verbesserung.

In [29]:
top_5_reduced_nodes = sorted(complete_graph.degree(reduced_node_list), key=lambda x: x[1], reverse=True)[:5]
print(top_5_reduced_nodes)

[('Betriebssystem', 825), ('Unicode', 728), ('Amsterdam', 671), ('MacOS', 638), ('Java (Programmiersprache)', 612)]


## Zusammenfassung Versuch 2
#### Man kann sehen, dass die reine Bedingung an den Knoten des Hauptgraphen noch nicht aussreicht für gute Ergebnisse. -> Zu oberflächliche Themen. Der Subgraph hingegen, schlägt zu spezifische Artikel vor.  

In [30]:
# export the subgraph to a csv file
edges = subgraph.edges
with open('edges_reduced.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

# Versuch 3: Extrahiere Knoten, die zu Python verlinkt sind
#### Hier wird die Bedingung getestet, dass nur die Knoten betrachtet werden die zu Python verlinkt sind.

#### Ebenfalls werde ich dies hier am Hauptgraphen und an einem Subgraphen testen.

In [31]:
nodes_with_edge_to_article = [node for node in complete_graph.nodes() if complete_graph.has_edge(node, 'Python (Programmiersprache)')]

## Mit Ursprungsgaphen
#### Es zeigt sich, dass wir ein durchwegs gute Themenvorschläge erhalten.

In [32]:
top_5_linked = sorted(complete_graph.degree(nodes_with_edge_to_article), key=lambda x: x[1], reverse=True)[:5]
print(top_5_linked)

[('Linux', 657), ('R (Programmiersprache)', 657), ('MacOS', 638), ('Java (Programmiersprache)', 612), ('C++', 567)]


## Mit Subgraphen
#### Deutlich erkennt man, dass die Themenvorschläge zu spezifisch sind.

In [33]:
subgraph_linked = complete_graph.subgraph(nodes_with_edge_to_article)
top_5_linked = sorted(subgraph_linked.degree(), key=lambda x: x[1], reverse=True)[:5]
print(top_5_linked)

[('Softwareentwickler', 428), ('C++', 401), ('Linux', 384), ('Java (Programmiersprache)', 383), ('C (Programmiersprache)', 377)]


In [34]:
# export the subgraph to a csv file
edges = subgraph_linked.edges
with open('edges_linked.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

## Zusammenfassung Versuch 3
#### Diese Bedingung sorgt für signifikant bessere Vorschläge am Hauptgraphen. Der Subgraph hingegen, schlägt zu spezifische Artikel vor.

# Versuch 4: Extrahiere Knoten mit beiden Bedingungen
#### Zum Schluss setzten wir beide Bedingungen vorraus. Auch hier wird dies am Hauptgraphen und an einem Subgraphen getestet.

In [35]:
nodes_with_edge_to_article_reduced = [node for node in reduced_node_list if complete_graph.has_edge(node, 'Python (Programmiersprache)')]
#print(nodes_with_edge_to_bienen_reduced)

## Extraktion mit beiden Bedingungen am Hauptgraphen
#### Hier wird die Kombination von beiden Bedingungen am Hauptgraphen getestet. Diese Erbnisse unterschieden sich jedoch nicht von den Ergebnissen des dritten Versuchs.

In [36]:
top_5_clean = sorted(complete_graph.degree(nodes_with_edge_to_article_reduced), key=lambda x: x[1], reverse=True)[:5]
print(top_5_clean)

[('MacOS', 638), ('Java (Programmiersprache)', 612), ('C++', 567), ('C (Programmiersprache)', 527), ('Perl (Programmiersprache)', 478)]


## Erstellung und Extraktion mit beiden Bedingungen an eines Subgraphen
#### Dieser Subgraph enthält nur die Knoten, die beide Bedingungen.
#### Hier sind die ebenso wie in Versuch 3 die Themenvorschläge identisch. Das Einzige worin sich die Ergebnisse unterscheiden, ist eine abnahme im Degree.

In [37]:
subgraph_clean = complete_graph.subgraph(nodes_with_edge_to_article_reduced)

In [38]:
top_5_clean = sorted(subgraph_clean.degree(nodes_with_edge_to_article_reduced), key=lambda x: x[1], reverse=True)[:5]
print(top_5_clean)

[('C++', 75), ('Java (Programmiersprache)', 72), ('C (Programmiersprache)', 62), ('Ruby (Programmiersprache)', 54), ('Perl (Programmiersprache)', 52)]


In [39]:
# export the subgraph to a csv file
edges = subgraph_clean.edges
with open('edges_predecessors.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

## Zusammenfassung Versuch 4:
#### Auch hier schneidet der Hauptgraph besser ab als der Subgraph.    

# Fazit
#### Die besten Ergebnissen wurden am Hauptgraphen mit der Bedingung erzielt, dass nur die Knoten betrachtet werden, die zu Bienen verlinkt sind. Dass beide Bedungungen erfüllt sein müssen, hat keinen Unterschied gemacht.
#### Der Subgraph hat bei allen Versuchen zu spezifische Artikel vorgeschlagen. Dies könnte man damit erklären, dass Informationen vom Hauptgraphen verloren gehen, da nur ein Teil der Knoten betrachtet wird. Dadurch fehlen Ingoing und Outgoing Links, die den Degree beeinflussen.   

In [40]:
backlinks = python_page.backlinks


In [41]:
top_x_backlinks = sorted(backlinks, key=lambda x: x[1], reverse=True)[:5]
print(top_x_backlinks)

['O’Reilly Open Source Convention', 'Kürzester Pfad', 'Führende Null und Füllnull', 'Höhere Programmiersprache', 'Nächste-Nachbarn-Klassifikation']


In [42]:
pr_result = nx.pagerank(complete_graph)

In [43]:
top_n_pr = sorted(pr_result, key=lambda x: pr_result[x], reverse=True)[:5]

In [44]:
print(top_n_pr)

['Python (Programmiersprache)', 'Programmiersprache', 'Betriebssystem', 'Version (Software)', 'Softwareentwickler']


In [45]:
filtered_pr = nx.pagerank(graph)
top_n_filtered_pr = sorted(filtered_pr, key=lambda x: filtered_pr[x], reverse=True)[:5]
print(top_n_filtered_pr)

['Programmiersprache', 'Python (Programmiersprache)', 'Betriebssystem', 'Version (Software)', 'C (Programmiersprache)']


# Evaluation

Hier wird mit den Artikel "Katzen" getestet, ob die Vorschläge thematisch passen.


In [46]:
cat_page = wiki.page('Katzen')

In [47]:
katzen_graph = make_graph_optimized(cat_page)

In [48]:
katzen_backgraph = make_backgraph(cat_page)

In [49]:
print(len(cat_page.links), len(cat_page.backlinks), (len(cat_page.backlinks)/len(cat_page.links))) 

236 811 3.4364406779661016


In [50]:
katzen_complete_graph = nx.compose(katzen_graph, katzen_backgraph)

Evaluation 1: Extrahiere die Knoten mit dem höchsten Grad

In [51]:
top_5_cat_nodes = sorted(katzen_complete_graph.degree, key=lambda x: x[1], reverse=True)[:5]
print(top_5_cat_nodes)

[('Euro-Umlaufmünzen-Motivliste', 1772), ('Liste fiktionaler Tiere', 1357), ('Kanada', 1300), ('Katzen', 1292), ('Hauskatze', 796)]


Evaluation 2: Extrahiere die direkten Nachfolger 

In [52]:
links_katze = get_links(cat_page)
reduced_node_cat_list = [node for node in katzen_complete_graph.nodes() if node in links_katze]

In [53]:
# subgraph
subgraph_katze = katzen_complete_graph.subgraph(reduced_node_cat_list)
top_5_reduced_cat_nodes = sorted(subgraph_katze.degree(), key=lambda x: x[1], reverse=True)[:5]
print(top_5_reduced_cat_nodes)

[('Gemeinsame Normdatei', 126), ('Raubtiere', 119), ('Kleinkatzen', 118), ('Systematik (Biologie)', 101), ('Hauskatze', 101)]


In [54]:
# graph
top_5_reduced_cat_nodes = sorted(katzen_complete_graph.degree(reduced_node_cat_list), key=lambda x: x[1], reverse=True)[:5]
print(top_5_reduced_cat_nodes)

[('Kanada', 1300), ('Hauskatze', 796), ('Peru', 674), ('Säugetiere', 670), ('Schottland', 655)]


Evaluation 3: Extrahiere Knoten, die zu Katzen verlinkt sind

In [55]:
nodes_with_edge_to_katze = [node for node in katzen_complete_graph.nodes() if katzen_complete_graph.has_edge(node, 'Katzen')]

In [56]:
# graph
top_5_cat_linked = sorted(complete_graph.degree(nodes_with_edge_to_katze), key=lambda x: x[1], reverse=True)[:5]
print(top_5_cat_linked)

[('Hypertext Transfer Protocol', 163), ('Miozän', 2), ('Zunge', 2), ('Versuch und Irrtum', 2), ('Elster', 1)]


In [57]:
# subgraph
subgraph_linked_katze = katzen_complete_graph.subgraph(nodes_with_edge_to_katze)
top_5_cat_linked = sorted(subgraph_linked_katze.degree(), key=lambda x: x[1], reverse=True)[:5]
print(top_5_cat_linked)

[('Familie (Biologie)', 234), ('Raubtiere', 180), ('Hauskatze', 145), ('Katzenartige', 143), ('Säugetiere', 132)]


Evaluation 4: Extrahiere Knoten mit beiden Bedingungen

In [58]:
# graph
nodes_with_edge_to_katze_reduced = [node for node in reduced_node_cat_list if katzen_complete_graph.has_edge(node, 'Katzen')]
top_5_clean_cat = sorted(katzen_complete_graph.degree(nodes_with_edge_to_katze_reduced), key=lambda x: x[1], reverse=True)[:5]
print(top_5_clean_cat)

[('Hauskatze', 796), ('Säugetiere', 670), ('Urin', 472), ('Tiger', 368), ('Löwe', 361)]


In [59]:
# subgraph
subgraph_clean_katze = katzen_complete_graph.subgraph(nodes_with_edge_to_katze_reduced)
top_5_clean_cat = sorted(subgraph_clean_katze.degree(nodes_with_edge_to_katze_reduced), key=lambda x: x[1], reverse=True)[:5]
print(top_5_clean_cat)

[('Kleinkatzen', 109), ('Raubtiere', 82), ('Katzenartige', 79), ('Familie (Biologie)', 71), ('Hauskatze', 55)]


PageRank

In [60]:
filtered_pr_katze = nx.pagerank(katzen_graph)
top_5_filtered_pr_katze = sorted(filtered_pr_katze, key=lambda x: filtered_pr_katze[x], reverse=True)[:5]
print(top_5_filtered_pr_katze)

['Katzen', 'Gemeinsame Normdatei', 'Systematik (Biologie)', 'Raubtiere', 'Familie (Biologie)']


In [61]:
pr_result_katze = nx.pagerank(katzen_complete_graph)
top_5_pr_katze = sorted(pr_result_katze, key=lambda x: pr_result_katze[x], reverse=True)[:5]
print(top_5_pr_katze)

['Katzen', 'Systematik (Biologie)', 'Nomenklatur (Biologie)', 'Gattung (Biologie)', 'Familie (Biologie)']


In [62]:
article_list = ['Muay Thai', 'Hunde', 'Pferde']

In [66]:
with open('output.txt', 'w', encoding='utf-8') as f:
    for each in article_list:
        article = wiki.page(each)
        article_graph = make_graph_optimized(article)
        article_backgraph = make_backgraph(article)
        article_complete_graph = nx.compose(article_graph, article_backgraph)
        
        links_article = get_links(article)
        reduced_node_list = [node for node in complete_graph.nodes() if node in links_article]
        
        f.write("Article: " + article.title + "\n")
        
        # V1
        top_5_article_nodes = sorted(article_complete_graph.degree, key=lambda x: x[1], reverse=True)[:5]
        f.write("V1\n")
        f.write(str(top_5_article_nodes) + "\n")
        
        # V2
        top_5_reduced_nodes = sorted(article_complete_graph.degree(reduced_node_list), key=lambda x: x[1], reverse=True)[:5]
        f.write("V2\n")
        f.write(str(top_5_reduced_nodes) + "\n")
        
        f.write("Subgraph\n")
        subgraph = article_complete_graph.subgraph(reduced_node_list)
        top_5_reduced_nodes = sorted(subgraph.degree(), key=lambda x: x[1], reverse=True)[:5]
        f.write(str(top_5_reduced_nodes) + "\n")
        
        # V3
        nodes_with_edge_to_article = [node for node in article_complete_graph.nodes() if article_complete_graph.has_edge(node, article.title)]
        top_5_linked = sorted(article_complete_graph.degree(nodes_with_edge_to_article), key=lambda x: x[1], reverse=True)[:5]
        f.write("V3\n")
        f.write(str(top_5_linked) + "\n")
        
        f.write("Subgraph\n")
        subgraph_linked = article_complete_graph.subgraph(nodes_with_edge_to_article)
        top_5_linked = sorted(subgraph_linked.degree(), key=lambda x: x[1], reverse=True)[:5]
        f.write(str(top_5_linked) + "\n")
        
        # V4
        nodes_with_edge_to_article_reduced = [node for node in reduced_node_list if article_complete_graph.has_edge(node, article.title)]
        top_5_clean = sorted(article_complete_graph.degree(nodes_with_edge_to_article_reduced), key=lambda x: x[1], reverse=True)[:5]
        f.write("V4\n")
        f.write(str(top_5_clean) + "\n")
        
        f.write("Subgraph\n")
        subgraph_clean = article_complete_graph.subgraph(nodes_with_edge_to_article_reduced)
        top_5_clean = sorted(subgraph_clean.degree(nodes_with_edge_to_article_reduced), key=lambda x: x[1], reverse=True)[:5]
        f.write(str(top_5_clean) + "\n")
        
        # PageRank
        f.write("PageRank\n")
        pr_result = nx.pagerank(article_complete_graph)
        top_n_pr = sorted(pr_result, key=lambda x: pr_result[x], reverse=True)[:5]
        f.write(str(top_n_pr) + "\n")
        
        f.write("Filtered PageRank\n")
        filtered_pr = nx.pagerank(article_graph)
        top_n_filtered_pr = sorted(filtered_pr, key=lambda x: filtered_pr[x], reverse=True)[:5]
        f.write(str(top_n_filtered_pr) + "\n")
        
        f.write("Link Faktor: " + str(len(article.links)) + ", " + str(len(article.backlinks)) + ", " + str((len(article.backlinks)/len(article.links))) + "\n")